# 해시법 - 오픈 주소법
자료를 추가 및 삭제, 검색 할 때 해시법을 사용하는 경우가 있음. 
해시법은 정해진 용량에 key-value로 일대일 대응된 자료를 저장할 때 주로 사용하는데, 일반적인(?) 해시법은 같은 해시값을 가진 key에 충돌 문제를 일으킴. 이때 이 충돌을 해결하려는 방안으로 체인법과 오픈주소법을 고려할 수 있는데, 체인법은 다른 코드를 참조하고 이 노트에서는 오픈 주소법을 다룸

- 참고로 해시는 파이썬의 딕셔너리와 같음  

## 오픈 주소법
- 해시값 충돌이 발생했을 때 재해시(rehashing)하는 방법으로 빈 버킷을 찾는 과정을 재해시라고 함. 
- 재해시를 하는 방법은 여러가지가 있으며, 여기서는 선형 탐사법을 이용하여 재해시 함
- 체인법에서는 같은 해시값을 가진 key값을 연결 리스트로 구현했다는 점에서 차이가 있음.

# 구현

`OpenHash` Class 
- 검색 `search`
 - key에 해당하는 해시값을 구하고, 해당하는 bucket에 접근한다
 - 해당하는 bucket의 key와 찾고자 하는 key를 비교
 - 경우의수 1) 버킷이 비어 있으면 -> 일치하는 key가 없는 것임 (검색 실패).
 - 경우의수 2) 삭제 완료 or 이미 있음 ->  값을 비교
 - 경우의수 2-1) 재해시하면서 일치하는 key를 찾음 (검색 성공)
 - 경우의수 2-2) 재해시하지만 비어있는 버킷을 봄 (검색 실패)
 - 경우의수 2-3) 재해시하지만 다 차있거나, 삭제 완료이지만 일치하는 key가 없음 (검색 실패)
 
- 추가 `add`
 - 이미 같은 key를 가진 버킷이 존재하면 추가 실패
 - 해시값과 그에 대응하는 버킷 주목
 - 주목된 버킷에 이미 값이 있다면, 재해시
 - 빈 버킷에 추가
 - 꽉 찼다면 추가 실패
 
- 제거 `remove`
 - 같은 key값을 가진 버킷이 존재하지 않으면 제거 실패
 - 같은 key값을 가진 버킷을 선형탐색
 - 제거

- 출력 `dump`

In [59]:
import hashlib
from typing import Any, Type

# 버킷의 상태 클래스
class Status:
    
    OCCUPIED = 0
    EMPTY = 1
    DELETED = 2

# 버킷 클래스
class Bucket:
    
    def __init__(self, key: Any = None, value: Any = None, stat: Status = Status.EMPTY):
        self.key = key
        self.value = value
        self.stat = stat
    
    
    def set(self, key, value, stat: Status):
        self.key = key
        self.value = value
        self.stat = stat
        
    def set_status(self, stat: Status):
        self.stat = stat
        

        

# 선형 검색을 이용한 해시
class OpenHash:
    
    # 초기화
    def __init__(self, length):
        self.length = length
        self.hash_table = [Bucket()] * self.length # 버킷의 초기 상태: all 비어있음(EMPTY)
        
    # 해시값
    def hash_value(self, key):
        self.key = key
        if isinstance(self.key, int):
            return self.key % self.length
        else:
            return int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % self.length
    
    # 선형 탐사를 위한 재해시값
    def rehash_value(self,key):
        return (self.hash_value(key) + 1) % self.length 
    
    # 검색
    def search(self, key):
        hash_value = self.hash_value(key) # 해시값 저장
        bucket = self.hash_table[hash_value] # 맨처음 볼 버킷
        
        for i in range(self.length):
            if key == bucket.key:
                return bucket
            
            if bucket.stat == Status.EMPTY:
                return False
                
            hash_value = self.rehash_value(hash_value) # 선형탐색
            bucket = self.hash_table[hash_value] # 재해시
            
    def search_value(self, key):
        bucket = self.search(key)
        if bucket != False:
            return bucket.value
        
        return None
            
    # 추가
    def add(self,key,value):
      
        if self.search(key) != False: # 동일한 key값이 존재하지 않는 경우에 추가할 수 있음
            return False

        hash_value = self.hash_value(key) # 해시값
        bucket = self.hash_table[hash_value] # 해시값에 해당하는 버킷
        
        for i in range(self.length):
            if bucket.stat == Status.EMPTY or bucket.stat == Status.DELETED: # 버킷이 비어있다면 추가
                self.hash_table[hash_value] = Bucket(key, value, Status.OCCUPIED)
                return True
            
            hash_value = self.rehash_value(key) # 안비어있으면 재해시
            bucket = self.hash_table[hash_value]
            
        return False
    
    # 제거
    def remove(self,key):
        bucket = self.search(key)
        if bucket == False: # Fasle -> 동일한 key값이 없음 -> 제거 불가
            return False
        
        bucket.set(None, None, Status.DELETED)
        return True    
    

In [60]:
openhash = OpenHash(5)

In [61]:
openhash.add(5,10)

True

In [62]:
openhash.search_value(5)

10

In [63]:
openhash.add(5,11)

False

In [64]:
openhash.remove(5)

True

In [65]:
openhash.search_value(5)

`OpenHash` Class 
- 버킷 검색 `search_bucket`
 - key에 해당하는 해시값을 구하고, 해당하는 bucket에 접근한다
 - 해당하는 bucket의 key와 찾고자 하는 key를 비교
 - 경우의수 1) 버킷이 비어 있으면 -> 일치하는 key가 없는 것임 (검색 실패).
 - 경우의수 2) 삭제 완료 or 이미 있음 ->  값을 비교
 - 경우의수 2-1) 재해시하면서 일치하는 key를 찾음 (검색 성공)
 - 경우의수 2-2) 재해시하지만 비어있는 버킷을 봄 (검색 실패)
 - 경우의수 2-3) 재해시하지만 다 차있거나, 삭제 완료이지만 일치하는 key가 없음 (검색 실패)
 
- 밸류 검색 `search_value`
 - `search_bucket`에서 구한 버킷이 차 있으면 그 버킷의 value 출력
 
- 추가 `add`
 - 이미 같은 key를 가진 버킷이 존재하면 추가 실패
 - 해시값과 그에 대응하는 버킷 주목
 - 주목된 버킷에 이미 값이 있다면, 재해시
 - 빈 버킷에 추가
 - 꽉 찼다면 추가 실패
 
- 제거 `remove`
 - 같은 key값을 가진 버킷이 존재하지 않으면 제거 실패
 - 같은 key값을 가진 버킷을 선형탐색
 - 제거

- 출력 `dump`

In [67]:
import hashlib

class Status:
    
        EMPTY = 0
        OCCUPIED = 1
        DELETED = 2

        
class Bucket:
    
    def __init__(self, key = None, value = None, stat = Status.EMPTY):
        self.key = key
        self.value = value
        self.stat = stat
        
    def set(self, key, value, stat: Status):
        self.key = key
        self.value = value
        self.stat = stat
        

class OpenHash:
    
    # 빈 버킷 생성
    def __init__(self, length):
        self.length = length
        self.table = [Bucket()] * self.length
        
    # 해시값 구하기
    def hash(self, key):
        if isinstance(key, int):
            return key % self.length
        else:
            return int( hashlib.sha256(str(key).encode()).hexdigest(), 16)  % self.length
        
    # 재해시
    def rehash(self, key):
        return (key+1) % self.length
    
    # 일치하는 key를 담은 버킷 검색
    def search_bucket(self, key):
        
        # 해시값과 해시값에 대응하는 첫 버킷
        hash = self.hash(key)
        bucket = self.table[hash]
        
        # 전체 탐색 (중간에 찾으면 종료함)
        for i in range(self.length):
            if bucket.stat == Status.EMPTY:
                return False
            
            # key와 일치하면 그 키를 담고 있는 버킷 출력
            if bucket.key == key and bucket.stat == Status.OCCUPIED:
                return bucket
            
            # 재해시
            hash = self.rehash(hash)
            bucket = self.table[hash]
        
        # 완전 탐색했음에도 일치하는 버킷 없음
        return False
            
            
    
    # 일치하는 key에 대응하는 value 검색
    def search_value(self, key):
        bucket = self.search_bucket(key)
        
        if bucket == False:
            return False
        
        return bucket.value
        
    # 추가
    def add(self,key,value):
        
        hash = self.hash(key)
        bucket_check = self.search_bucket(key)
        bucket = self.table[hash]
            
        
        # 일치하는 key가 없거나 삭제된 경우에만 추가 가능 
        if bucket_check == False:
            for i in range(self.length):
                if bucket.stat == Status.EMPTY or bucket.stat == Status.DELETED:
                    self.table[hash] = Bucket(key,value,Status.OCCUPIED)
                    return True
                else:
                    # 이미 OCCUPIED된 상태이면 빈 공간 찾을 때 까지 재해시
                    hash = self.rehash(hash)
                    bucket = self.table[hash]
            # 완전 탐색했는데도 추가 실패 = 꽉 차 있음
            return "FULL"
    
    #제거    
    def remove(self,key):
        bucket = self.search_bucket(key)
        if bucket != False:
            bucket.set(bucket.key, bucket.value, Status.DELETED)
            return True
        return False
    
    # 출력
    def dump(self):
        for i in range(self.length):
            bucket =  self.table[i]
            if bucket.stat == Status.OCCUPIED:
                print(f"{i} | {bucket.key} : {bucket.value}")
                
            elif bucket.stat == Status.DELETED:
                print(f"{i} | {bucket.key} : {bucket.value} - 삭제됨")
            elif bucket.stat == Status.EMPTY:
                print(f"{i} | -----미등록-----")

In [68]:
openhash = OpenHash(5)

In [69]:
openhash.search_bucket("경현")

False

In [70]:
openhash.add(1,3)

True

In [71]:
openhash.add("경현",3)

True

In [72]:
openhash.dump()

0 | -----미등록-----
1 | 1 : 3
2 | -----미등록-----
3 | 경현 : 3
4 | -----미등록-----


In [73]:
openhash.search_value("경현")

3

In [74]:
openhash.remove("경현")

True

In [75]:
openhash.search_value("경현")

False

In [76]:
openhash.dump()

0 | -----미등록-----
1 | 1 : 3
2 | -----미등록-----
3 | 경현 : 3 - 삭제됨
4 | -----미등록-----
